# Assignment 1: Wrangling and EDA
### Foundations of Machine Learning

In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `airbnb_NYC.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)

To clean the 'Price' variable, I first converted everything into a string and removed formatting issues like dollar signs and commas. A value like 1,001 would have the comma removed. I also stripped extra spaces and treated blank or invalid entries as missing. After cleaning, I converted the values to numeric, changing any non-convertible values to become NaN. We end with 0 missing values. 

2. Categorical variable: For the Minnesota police use of for data, `mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing? For the remaining missing values, replace the `np.nan/None` values with the label `Missing`.

For subject_injury, I standardized all text by trimming whitespace and converting it to lowercase, then grouped responses into Yes or No categories based on keywords such as “injured” or “no injury”. I treated ambiguous responses like “unknown” or “n/a” as missing values. Also, I calculated the proportion of missing values and examined how missingness varies across force_type using a cross-tab and a missing-rate summary. After analyzing the pattern, I replaced the remaining missing values with the label “Missing” so they could still be included in summaries.

3. Dummy variable: For `metabric.csv`, convert the `Overall Survival Status` variable into a dummy/binary variable, taking the value 0 if the patient is deceased and 1 if they are living.

I created a binary survival variable from Overall Survival Status by searching for keywords that indicate whether a patient was alive or dead. If the status contained terms like “living” or “alive” I assigned a 1 while if it contained terms like “deceased” “dead” I assigned a 0. Any entries that did not fall into one of these categories were left as missing to avoid misclassification.

4. Missing values: For `airbnb_NYC.csv`, determine how many missing values of `Review Scores Rating` there are. Create a new variable, in which you impute the median score for non-missing observations to the missing ones. Why might this bias or otherwise negatively impact your results?

I counted the number of missing values in Review Scores Rating and then I created a new variable where missing ratings were filled with the median of the observed ratings. Using the median helps reduce the impact of extreme values, but this method can still introduce somes biases. It reduces variation in the data and assumes that missing ratings are similar to the typical listing, which may not be true if missingness is not random and has a pattern.

In [9]:
airbnb_df = pd.read_csv("/Users/timothylee/DS Analytics/ml_container/data/airbnb_NYC.csv", encoding="latin1")
police_df = pd.read_csv("/Users/timothylee/DS Analytics/ml_container/data/mn_police_use_of_force.csv", encoding="latin1")
metabric_df = pd.read_csv("/Users/timothylee/DS Analytics/ml_container/data/metabric.csv", encoding="latin1")

price_raw = airbnb_df["Price"].astype(str).str.strip()

price_cleaned = (
    price_raw
    .replace({"": np.nan, "nan": np.nan, "None": np.nan})
    .str.replace(r"[\$,]", "", regex=True)
)

airbnb_df["price_numeric"] = pd.to_numeric(price_cleaned, errors="coerce")

print("Q1.1 Missing after cleaning:", airbnb_df["price_numeric"].isna().sum())

inj_raw = police_df["subject_injury"].astype(str).str.strip().str.lower()

missing_vals = {
    "", "nan", "none", "null", "na",
    "n/a", "unknown", "unsure", "undetermined"
}

inj_raw = inj_raw.where(~inj_raw.isin(missing_vals), np.nan)

yes_values = {"yes", "y", "injured", "injury", "true", "1"}
no_values  = {"no", "n", "not injured", "no injury", "false", "0"}

def recode_injury(val):
    if pd.isna(val):
        return np.nan
    if val in yes_values:
        return "Yes"
    if val in no_values:
        return "No"
    return np.nan

police_df["injury_clean"] = inj_raw.map(recode_injury)

print("Q1.2 Missing proportion:", police_df["injury_clean"].isna().mean())

print("\nQ1.2 Crosstab (before labeling Missing):")
print(pd.crosstab(police_df["force_type"],
                  police_df["injury_clean"],
                  dropna=False))

police_df["inj_missing_flag"] = police_df["injury_clean"].isna()

print("\nMissing rate by force_type:")
print(police_df.groupby("force_type")["inj_missing_flag"]
      .mean()
      .sort_values(ascending=False))

police_df["injury_clean"] = police_df["injury_clean"].fillna("Missing")

print("\nCrosstab after labeling Missing:")
print(pd.crosstab(police_df["force_type"],
                  police_df["injury_clean"],
                  margins=True))

surv_raw = metabric_df["Overall Survival Status"].astype(str).str.strip().str.lower()

def make_binary(status):
    if pd.isna(status) or status in {"", "nan", "none", "na", "n/a"}:
        return np.nan
    if "deceased" in status or "dead" in status or "died" in status:
        return 0
    if "living" in status or "alive" in status:
        return 1
    if status in {"0", "1"}:
        return int(status)
    return np.nan

metabric_df["survival_binary"] = surv_raw.map(make_binary)

print("\nQ1.3 Dummy counts:")
print(metabric_df["survival_binary"].value_counts(dropna=False))
print("Missing in dummy:", metabric_df["survival_binary"].isna().sum())

airbnb_df["Review Scores Rating"] = pd.to_numeric(
    airbnb_df["Review Scores Rating"],
    errors="coerce"
)

print("\nQ1.4 Missing review scores:",
      airbnb_df["Review Scores Rating"].isna().sum())

median_score = airbnb_df["Review Scores Rating"].median(skipna=True)

airbnb_df["review_imputed"] = (
    airbnb_df["Review Scores Rating"]
    .fillna(median_score)
)

print("Median used:", median_score)
print("Missing after imputation:",
      airbnb_df["review_imputed"].isna().sum())

Q1.1 Missing after cleaning: 0
Q1.2 Missing proportion: 0.7619342359767892

Q1.2 Crosstab (before labeling Missing):
injury_clean                   No   Yes   NaN
force_type                                   
Baton                           0     2     2
Bodily Force                 1093  1286  7051
Chemical Irritant             131    41  1421
Firearm                         2     0     0
Gun Point Display              33    44    27
Improvised Weapon              34    40    74
Less Lethal                     0     0    87
Less Lethal Projectile          1     2     0
Maximal Restraint Technique     0     0   170
Police K9 Bite                  2    44    31
Taser                         150   172   985

Missing rate by force_type:
force_type
Less Lethal                    1.000000
Maximal Restraint Technique    1.000000
Chemical Irritant              0.892028
Taser                          0.753634
Bodily Force                   0.747720
Baton                          0.500000
Impro

**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks.

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work. What does work?
2. Drop any columns that do not contain data.
3. What is an observation? Carefully justify your answer, and explain how it affects your choices in cleaning and analyzing the data.
4. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
5. Clean the Age variable and make a histogram of the ages of the victims.
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Thoughts?

In [ ]:
shark_df = pd.read_excel("GSAF5.xls", engine="xlrd")
shark_df = shark_df.dropna(axis=1, how="all")

year_text = shark_df["Year"].astype(str)
year_digits = year_text.str.extract(r"(\d{4})", expand=False)
shark_df["year_num"] = pd.to_numeric(year_digits, errors="coerce")

print("Cleaned year range:",
      shark_df["year_num"].min(skipna=True),
      "to",
      shark_df["year_num"].max(skipna=True))

recent_df = shark_df[shark_df["year_num"] >= 1940].copy()

year_counts = (
    recent_df
    .groupby("year_num")
    .size()
    .sort_index()
)

x_vals = year_counts.index.to_numpy()
y_vals = year_counts.values

trend_coef = np.polyfit(x_vals, y_vals, 1)[0] if len(x_vals) > 1 else np.nan
print("Trend slope (attacks per year):", trend_coef)

plt.figure()
plt.plot(x_vals, y_vals)
plt.title("Shark Attacks per Year (1940+)")
plt.xlabel("Year")
plt.ylabel("Number of Attacks")
plt.show()

age_raw = (
    recent_df["Age"]
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({"": np.nan, "nan": np.nan, "none": np.nan})
)

def process_age(val):
    if pd.isna(val):
        return np.nan

    val = str(val).strip().lower()

    match_range = pd.Series([val]).str.extract(r"^(\d+)\s*-\s*(\d+)$")
    if not match_range.isna().all(axis=None):
        low = float(match_range.iloc[0, 0])
        high = float(match_range.iloc[0, 1])
        return (low + high) / 2

    match_single = pd.Series([val]).str.extract(r"^(\d+)$")
    if not match_single.isna().all(axis=None):
        return float(match_single.iloc[0, 0])

    match_any = pd.Series([val]).str.extract(r"(\d+)")
    if not match_any.isna().all(axis=None):
        return float(match_any.iloc[0, 0])

    return np.nan

recent_df["age_num"] = age_raw.map(process_age)

plt.figure()
plt.hist(recent_df["age_num"].dropna(), bins=30)
plt.title("Victim Age Distribution (1940+)")
plt.xlabel("Age")
plt.ylabel("Frequency")
plt.show()

type_raw = (
    recent_df["Type"]
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({"": np.nan, "nan": np.nan, "none": np.nan})
)

def categorize_type(entry):
    if pd.isna(entry):
        return "Unknown"

    entry = str(entry).lower()

    if "unprovoked" in entry:
        return "Unprovoked"
    if "provoked" in entry:
        return "Provoked"

    return "Unknown"

recent_df["type_group"] = type_raw.map(categorize_type)

prop_unprovoked = (recent_df["type_group"] == "Unprovoked").mean()
print("Share of unprovoked attacks (since 1940):", prop_unprovoked)

fatal_raw = (
    recent_df["Fatal Y/N"]
    .astype(str)
    .str.strip()
    .str.upper()
    .replace({"": np.nan, "NAN": np.nan, "NONE": np.nan})
)

def standardize_fatal(flag):
    if pd.isna(flag):
        return "Unknown"

    flag = str(flag).upper()

    if flag.startswith("Y"):
        return "Y"
    if flag.startswith("N"):
        return "N"

    return "Unknown"

recent_df["fatal_flag"] = fatal_raw.map(standardize_fatal)

print(pd.crosstab(recent_df["type_group"],
                  recent_df["fatal_flag"],
                  margins=True))

fatal_rates = (
    recent_df
    .assign(fatal_binary=recent_df["fatal_flag"] == "Y")
    .groupby("type_group")["fatal_binary"]
    .mean()
)

print("\nFatality rate by type:")
print(fatal_rates)


ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 2.0.1 for xls Excel support Use pip or conda to install xlrd.

**Q3.** Open the "tidy_data.pdf" document available in `https://github.com/ds4e/wrangling`, which is a paper called *Tidy Data* by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?

**Q4.** This question looks at financial transfers from international actors to American universities. In particular, from which countries and giftors are the gifts coming from, and to which institutions are they going? 

For this question, `.groupby([vars]).count()` and `.groupby([vars]).sum()` will be especially useful to tally the number of occurrences and sum the values of those occurrences.

1. Load the `ForeignGifts_edu.csv` dataset.
2. For `Foreign Gift Amount`, create a histogram and describe the variable. Describe your findings.
3. For `Gift Type`, create a histogram or value counts table. What proportion of the gifts are contracts, real estate, and monetary gifts?
4. What are the top 15 countries in terms of the number of gifts? What are the top 15 countries in terms of the amount given?
5. What are the top 15 institutions in terms of the total amount of money they receive? Make a histogram of the total amount received by all institutions. 
6. Which giftors provide the most money, in total? 

**Q5.** This question provides some practice doing exploratory data analysis and visualization.

We'll use the `college_completion.csv` dataset from the US Department of Education. The "relevant" variables for this question are:
  - `level` - Level of institution (4-year, 2-year)
  - `aid_value` - The average amount of student aid going to undergraduate recipients
  - `control` - Public, Private not-for-profit, Private for-profit
  - `grad_100_value` - percentage of first-time, full-time, degree-seeking undergraduates who complete a degree or certificate program within 100 percent of expected time (bachelor's-seeking group at 4-year institutions)

1. Load the `college_completion.csv` data with Pandas.
2. How many observations and variables are in the data? Use `.head()` to examine the first few rows of data.
3. Cross tabulate `control` and `level`. Describe the patterns you see in words.
4. For `grad_100_value`, create a kernel density plot and describe table. Now condition on `control`, and produce a kernel density plot and describe tables for each type of institutional control. Which type of institution appear to have the most favorable graduation rates?
5. Make a scatterplot of `grad_100_value` by `aid_value`, and compute the covariance and correlation between the two variables. Describe what you see. Now make the same plot and statistics, but conditioning on `control`. Describe what you see. For which kinds of institutions does aid seem to vary positively with graduation rates?

**Q6.** In class, we talked about how to compute the sample mean of a variable $X$,
$$
m(X) = \dfrac{1}{N} \sum_{i=1}^N x_i
$$
and sample covariance of two variables $X$ and $Y$,
$$
\text{cov}(X,Y) = \dfrac{1}{N} \sum_{i=1}^N (x_i - m(X))(y_i - m(Y))).
$$
Recall, the sample variance of $X$ is
$$
s^2 = \dfrac{1}{N} \sum_{i=1}^N (x_i - m(X))^2.
$$
It can be very helpful to understand some basic properties of these statistics. If you want to write your calculations on a piece of paper, take a photo, and upload that to your GitHub repo, that's probably easiest.

We're going to look at **linear transformations** of $X$, $Y = a + bX$. So we take each value of $X$, $x_i$, and transform it as $y_i = a + b x_i$. 

1. Show that $m(a + bX) = a+b \times m(X)$.
2. Show that $ \text{cov}(X,X) = s^2$.
3. Show that $\text{cov}(X,a+bY) = b \times \text{cov}(X,Y)$
4. Show that $\text{cov}(a+bX,a+bY) = b^2 \text{cov}(X,Y) $. Notice, this also means that $\text{cov}(bX, bX) = b^2 s^2$.
5. Suppose $b>0$ and let the median of $X$ be $\text{med}(X)$. Is it true that the median of $a+bX$ is equal to $a + b \times \text{med}(X)$? Is the IQR of $a + bX$ equal to $a + b \times \text{IQR}(X)$?
6. Show by example that the means of $X^2$ and $\sqrt{X}$ are generally not $(m(X))^2$ and $\sqrt{m(X)}$. So, the results we derived above really depend on the linearity of the transformation $Y = a + bX$, and transformations like $Y = X^2$ or $Y = \sqrt{X}$ will not behave in a similar way.

**Q7.** This question provides some practice doing exploratory data analysis and visualization.

We'll use the `ames_prices.csv` dataset. The "relevant" variables for this question are:
  - `price` - Sale price value of the house
  - `Bldg.Type` - Building type of the house (single family home, end-of-unit townhome, duplex, interior townhome, two-family conversion)

1. Load the `college_completion.csv` data with Pandas.
2. Make a kernel density plot of price and compute a describe table. Now, make a kernel density plot of price conditional on building type, and use `.groupby()` to make a describe type for each type of building. Which building types are the most expensive, on average? Which have the highest variance in transaction prices?
3. Make an ECDF plot of price, and compute the sample minimum, .25 quantile, median, .75 quantile, and sample maximum (i.e. a 5-number summary).
4. Make a boxplot of price. Are there outliers? Make a boxplot of price conditional on building type. What patterns do you see?
5. Make a dummy variable indicating that an observation is an outlier.
6. Winsorize the price variable, and compute a new kernel density plot and describe table. How do the results change?